# Generación de texto con LSTMs

In [1]:
# Paso 1: importar librerías
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras import models, layers

Using TensorFlow backend.


In [2]:
#Paso 2: importar cuento  + pasar todo a minusculas + pasar caracteres a integers
# load ascii text and covert to lowercase
f = open('MobyDick.txt', 'r', encoding="utf8") 
text = f.read()
f.close()
text = text.lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = np.array(list(text))
encoder = preprocessing.LabelEncoder().fit(chars)
encoded_chars = encoder.transform(chars)

# normalize data
mean = encoded_chars.mean()
stdv = encoded_chars.std()
encoded_chars = (encoded_chars - mean) / stdv

In [3]:
#paso 3: preparar datos: 
# 1- preprocesar: ejemplo: CHAPT -> E; HAPTE -> R (pero con 100 caracteres)
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

X_train, y_train = split_sequence(encoded_chars, 100)

# 2- cambiar dataset para que la entrada sea válida: [Samples, time_steps,features]
n_samples = X_train.shape[0]
n_steps = X_train.shape[1] # 100
n_features = 1
X_train = X_train.reshape((n_samples, n_steps, n_features))

# 3-Normalizar
    # normalizamos todo después de usar label encoder

# prepare the dataset of input to output pairs encoded as integers

In [5]:
# Preparar datos de salida (one-hot encoder)
# one hot encode the output variable

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.40, random_state=42)

In [6]:
# Definimos modelo LSTM
lstm_model = models.Sequential([
    layers.LSTM(32, input_shape=X_train.shape[-2:]),
    layers.Dense(1)
])

lstm_model.compile(optimizer='rmsprop', loss='mse')



In [7]:
#Hacemos el FIT
history = lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Train on 742886 samples, validate on 495258 samples
Epoch 1/5
314784/742886 [===========>..................] - ETA: 12:18 - loss: 0.9151

KeyboardInterrupt: 

In [ ]:
#Generar datos caracteres
# pick a random seed
